<a href="https://colab.research.google.com/github/Christi049/grocery-saver-agent/blob/main/grocery_saver_agentoff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

file = json.load(open("Dataset.json", "r"))
print(file[1])

{'instruction': 'Suggest a weekly meal plan prioritizing ingredients that will expire soon, with three meals per day, varied dishes, and minimal extra shopping.', 'input': {'dietary_restriction': 'Vegetarian', 'allergens': 'None', 'ingredients': [{'name': 'Paneer', 'expiry': '2025-09-04'}, {'name': 'Bell peppers', 'expiry': '2025-09-05'}, {'name': 'Onions', 'expiry': '2025-09-10'}, {'name': 'Tomatoes', 'expiry': '2025-09-06'}, {'name': 'Yogurt', 'expiry': '2025-09-03'}, {'name': 'Milk', 'expiry': '2025-09-03'}]}, 'output': {'meals': {'Monday': {'breakfast': 'Lassi (Yogurt drink) and toast with butter', 'lunch': 'Kadai Paneer with roti', 'dinner': 'Leftover Kadai Paneer with rice'}, 'Tuesday': {'breakfast': 'Yogurt and fruit bowl', 'lunch': 'Paneer tikka masala with naan', 'dinner': 'Leftover Paneer Tikka Masala'}, 'Wednesday': {'breakfast': 'Milkshake with banana', 'lunch': 'Paneer bhurji with paratha', 'dinner': 'Paneer and bell pepper stir-fry with roti'}, 'Thursday': {'breakfast': '

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 22.7 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
!pip install -q huggingface_hub transformers gradio accelerate

In [ ]:
# Only if you want to include your existing examples
with open("Dataset.json", "r") as f:
    current_examples = json.load(f)

In [ ]:
ingredients = ["eggs", "cheese", "sausage", "spinach", "mushrooms"]
methods = ["scrambled", "grilled", "baked", "sautéed"]
meals = ["omelette", "wrap", "muffin", "skillet"]
dietary_options = ["None", "Vegetarian", "Vegan", "Gluten-Free"]
allergen_options = ["Nuts", "Gluten", "Dairy", "Soy"]

new_examples = []

for method in methods:
    for meal in meals:
        for i in ingredients:
            for j in ingredients:
                if i != j:
                    dietary = dietary_options[len(new_examples) % len(dietary_options)]
                    allergens = allergen_options[len(new_examples) % len(allergen_options)]

                    instruction = "Suggest creative breakfast options using items that will expire soon."
                    input_data = {
                        "dietary_restriction": dietary,
                        "allergens": allergens,
                        "ingredients": [
                            {"name": i, "expiry": "2025-09-03"},
                            {"name": j, "expiry": "2025-09-03"}
                        ]
                    }
                    output_data = {
                        "meals": {
                            "Monday": f"{method} {i} and {j} {meal}",
                            "Tuesday": f"{method} {j} and {i} {meal}",
                            "Wednesday": f"{method} {i} {meal} with {j}",
                            "Thursday": f"{method} {j} {meal} with {i}",
                            "Friday": f"{method} {i} and {j} {meal} wrap",
                            "Saturday": f"{method} {i} and {j} {meal} skillet",
                            "Sunday": f"{method} {i} and {j} {meal} muffin"
                        },
                        "shopping_list": ["Tomatoes", "Onions", "Bell peppers"]
                    }
                    new_examples.append({
                        "instruction": instruction,
                        "input": input_data,
                        "output": output_data
                    })

print(f"Generated {len(new_examples)} new examples")


Generated 320 new examples


In [ ]:
# Only if you loaded current_examples
combined_dataset = current_examples + new_examples

In [ ]:
# Save new dataset only
with open("dataset.json", "w") as f:
    json.dump(new_examples, f, indent=2)

# Save combined dataset (if combined)
with open("combined_dataset.json", "w") as f:
    json.dump(combined_dataset, f, indent=2)


In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.2: Fast Llama patching. Transformers: 4.56.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
import json
from datasets import Dataset

def format_prompt(example):
    return (
        f"### Instruction: {example['instruction']}\n"
        f"### Input: {json.dumps(example['input'], ensure_ascii=False)}\n"
        f"### Output: {json.dumps(example['output'], ensure_ascii=False)}<|endoftext|>"
    )

# Use your combined_dataset instead of 'file'
formatted_data = [format_prompt(item) for item in combined_dataset]

# Create a Hugging Face Dataset
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.9.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/441 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 441 | Num Epochs = 3 | Total steps = 168
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 97,255,424 of 3,310,005,248 (2.94% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,0.540100
50,0.180600
75,0.138500
100,0.121600
125,0.107900
150,0.091100


In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt (dataset format)
test_instruction = "Generate a 7-day meal plan with Breakfast, Lunch, and Dinner listed under each day. Prioritize ingredients nearing expiry, keep dishes varied, minimize extra shopping, and include both popular Western dishes available in India and Kerala dishes, while respecting dietary restrictions ({{dietary_restriction}}) and avoiding allergens ({{allergens}})."
test_input = {
    "dietary_restriction": "Vegan",
    "allergens": "Nuts",
    "ingredients": [
        {"name": "Zucchini", "expiry": "2025-08-08"},
        {"name": "Rice", "expiry": "2025-08-10"},
        {"name": "Curd", "expiry": "2025-08-22"}
    ]
}

messages = [
    {"role": "user", "content": f"### Instruction: {test_instruction}\n### Input: {json.dumps(test_input)}\n### Output:"}
]

# Step 1: Get chat-formatted text
chat_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,              # <-- don't tokenize yet
    add_generation_prompt=True
)

# Step 2: Tokenize with attention_mask
inputs = tokenizer(
    chat_text,
    return_tensors="pt",
    return_attention_mask=True,  # <-- ensures attention_mask is included
).to("cuda")

# Step 3: Generate response
outputs = model.generate(
    **inputs,                    # <-- now works, inputs is a dict
    max_new_tokens=512,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
response = response.split("<|endoftext|>")[0]
print(response)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 08 Sep 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

### Instruction: Generate a 7-day meal plan with Breakfast, Lunch, and Dinner listed under each day. Prioritize ingredients nearing expiry, keep dishes varied, minimize extra shopping, and include both popular Western dishes available in India and Kerala dishes, while respecting dietary restrictions ({{dietary_restriction}}) and avoiding allergens ({{allergens}}).
### Input: {"dietary_restriction": "Vegan", "allergens": "Nuts", "ingredients": [{"name": "Zucchini", "expiry": "2025-08-08"}, {"name": "Rice", "expiry": "2025-08-10"}, {"name": "Curd", "expiry": "2025-08-22"}]}
### Output:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

### Meal Plan: {"meals": {"Monday": {"breakfast": "Zucchini fritters (made with zucchini and rice flour)", "lunch": "Zucchini and rice stir-fry with spices", "d

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
adapter_path = "outputs/checkpoint-168"   # replace with your latest checkpoint

# Save tokenizer (it’s from the base model)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained("meal-gen-adapter")

# Just copy adapter folder (LoRA) for upload
!cp -r {adapter_path} meal-gen-adapter/


In [ ]:
!ls meal-gen-adapter


adapter_config.json	   checkpoint-168	    tokenizer.json
adapter_model.safetensors  special_tokens_map.json
chat_template.jinja	   tokenizer_config.json


In [ ]:
from transformers import AutoTokenizer

base_model_id = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained("meal-gen-adapter")


('meal-gen-adapter/tokenizer_config.json',
 'meal-gen-adapter/special_tokens_map.json',
 'meal-gen-adapter/chat_template.jinja',
 'meal-gen-adapter/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from huggingface_hub import create_repo

# This will create "meal-gen-adapter" under your account Christi049
create_repo("meal-gen-adapter", exist_ok=True)


RepoUrl('https://huggingface.co/Christi049/meal-gen-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='Christi049/meal-gen-adapter')

In [ ]:
from huggingface_hub import upload_folder

repo_id = "Christi049/meal-gen-adapter"  # replace with your HF username

upload_folder(
    folder_path="meal-gen-adapter",
    repo_id=repo_id,
    repo_type="model"
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...dapter/checkpoint-168/rng_state.pth: 100%|##########| 14.6kB / 14.6kB            

  ...apter/checkpoint-168/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

  ...ent/meal-gen-adapter/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

...

  ...en-adapter/checkpoint-168/scaler.pt: 100%|##########| 1.38kB / 1.38kB            

  ...adapter/checkpoint-168/optimizer.pt:   0%|          |  525kB /  198MB            

  ...adapter/checkpoint-168/scheduler.pt:   6%|5         |  84.0B / 1.47kB            

  ...er/checkpoint-168/training_args.bin:   6%|5         |   350B / 6.10kB            

CommitInfo(commit_url='https://huggingface.co/Christi049/meal-gen-adapter/commit/9bba53773e06529bdf18e72074649861272e6426', commit_message='Upload folder using huggingface_hub', commit_description='', oid='9bba53773e06529bdf18e72074649861272e6426', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Christi049/meal-gen-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='Christi049/meal-gen-adapter'), pr_revision=None, pr_num=None)

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)